# Assignment 03 - Basic RAG Architecture



## Load Environment Variables
All the environment variables, APIS keys etc are loaded in the `.env` file. Load it into memory.

In [1]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

print("ℹ️  Loaded environment variables from .env file")

ℹ️  Loaded environment variables from .env file


## Define Data Sources

Real Meta PDFs from their Investor Relations website

In [2]:
# Real Meta earnings data sources
data_sources = {
    "Q1": {
        "press_release": "https://s21.q4cdn.com/399680738/files/doc_financials/2025/q1/Meta-03-31-2025-Exhibit-99-1-Final.pdf",
        "presentation": "https://s21.q4cdn.com/399680738/files/doc_financials/2025/q1/Earnings-Presentation-Q1-2025-FINAL.pdf",
        "earnings_call": "https://s21.q4cdn.com/399680738/files/doc_financials/2025/q1/Transcripts/META-Q1-2025-Earnings-Call-Transcript-1.pdf",
        "followup_qa": "https://s21.q4cdn.com/399680738/files/doc_financials/2025/q1/Transcripts/META-Q1-2025-Follow-Up-Call-Transcript.pdf"
    },
    "Q2": {
        "press_release": "https://s21.q4cdn.com/399680738/files/doc_downloads/Meta-06-30-2025-Exhibit-99-1-FINAL.pdf",
        "presentation": "https://s21.q4cdn.com/399680738/files/doc_downloads/Earnings-Presentation-Q2-2025-FINAL.pdf",
        "earnings_call": "https://s21.q4cdn.com/399680738/files/doc_financials/2025/q2/META-Q2-2025-Earnings-Call-Transcript.pdf",
        "followup_qa": "https://s21.q4cdn.com/399680738/files/doc_financials/2025/q2/META-Q2-2025-Follow-Up-Call-Transcript.pdf"
    },
    "Q3": {
        "press_release": "https://s21.q4cdn.com/399680738/files/doc_financials/2025/q3/Meta-09-30-2025-Exhibit-99-1-Final.pdf",
        "presentation": "https://s21.q4cdn.com/399680738/files/doc_financials/2025/q3/Earnings-Presentation-Q3-2025-Final.pdf",
        "earnings_call": "https://s21.q4cdn.com/399680738/files/doc_financials/2025/q3/META-Q3-2025-Earnings-Call-Transcript.pdf",
        "followup_qa": "https://s21.q4cdn.com/399680738/files/doc_financials/2025/q3/META-Q3-2025-Follow-Up-Call-Transcript.pdf"
    },
    "Q4": {
        "press_release": "https://s21.q4cdn.com/399680738/files/doc_financials/2025/q4/Meta-12-31-2025-Exhibit-99-1-FINAL.pdf",
        "presentation": "https://s21.q4cdn.com/399680738/files/doc_financials/2025/q4/Earnings-Presentation-Q4-2025-FINAL.pdf",
        "earnings_call": "https://s21.q4cdn.com/399680738/files/doc_financials/2025/q4/META-Q4-2025-Earnings-Call-Transcript.pdf",
        "followup_qa": "https://s21.q4cdn.com/399680738/files/doc_financials/2025/q4/META-Q4-2025-Follow-Up-Call-Transcript.pdf"
    }
}

print("✅ Data sources loaded!")
print()
print("Available quarters: Q1, Q2, Q3, Q4")
print("Document types per quarter: press_release, presentation, earnings_call, followup_qa")
print()
print(f"Total documents to process: {len(data_sources) * 4} PDFs")

✅ Data sources loaded!

Available quarters: Q1, Q2, Q3, Q4
Document types per quarter: press_release, presentation, earnings_call, followup_qa

Total documents to process: 16 PDFs


## Initialize Wrappers

Initialize the LLM Wrappers / Agents

In [3]:
from openai import OpenAI
import chromadb

# Initialize OpenAI client
client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))

# Initialize ChromaDB client (in-memory database, no API keys needed)
chroma_client = chromadb.Client()

## Utility Functions

Helper functions for PDF fetching and chunk storage

In [4]:
# Import required libraries
import requests
from io import BytesIO
from PyPDF2 import PdfReader
import fitz  # PyMuPDF

def fetch_and_extract_pdf(pdf_url, timeout=10, return_type="Text"):
    """
    Fetch a PDF from a URL and extract all text
    
    Args:
        pdf_url: URL to the PDF file
        timeout: Request timeout in seconds
        return_pages: If True, return list of pages; if False, return combined text
    
    Returns:
        If return_pages=False: Extracted text (string) or None if error
        If return_pages=True: List of page texts or None if error
    """
    try:
        response = requests.get(pdf_url, timeout=timeout)
        response.raise_for_status()
        
        pdf_file = BytesIO(response.content)
        reader = PdfReader(pdf_file)
        
        if return_type == "Pages":
            pages = []
            for page in reader.pages:
                page_text = page.extract_text()
                if page_text and page_text.strip():
                    pages.append(page_text.strip())
            return pages if pages else None
        elif return_type == "Paragraphs":
            paragraphs = []
            for page in reader.pages:
                page_text = page.extract_text()
                if page_text:
                    for para in page_text.split("\n\n"):
                        if para.strip():
                            paragraphs.append(para.strip())
            return paragraphs if paragraphs else None
        else: # Text
            text = ""
            for page in reader.pages:
                page_text = page.extract_text()
                if page_text:
                    text += page_text + "\n"
            
            return text.strip()
    
    except Exception as e:
        print(f"   ⚠️  Error fetching PDF: {str(e)[:60]}")
        return None

def fetch_and_extract_section_from_pdf(pdf_url, timeout=10):
    """
    Fetch a PDF from a URL and extract a specific section using PyMuPDF
    
    Args:
        pdf_url: URL to the PDF file
        timeout: Request timeout in seconds
    
    Returns:
        Extracted section text (string) or None if error
    """
    try:
        doc = fitz.open(stream=requests.get(pdf_url, timeout=timeout).content, filetype="pdf")
        sections = []
        current_section = None
        
        for page_num, page in enumerate(doc):
             blocks = page.get_text("dict")["blocks"]
             
             for block in blocks:
                if "lines" not in block:
                    continue
                
                for line in block["lines"]:
                    line_text = ""
                    max_font_size = 0
                    is_bold = False
                    
                    for span in line["spans"]:
                        for span in line["spans"]:
                            text = span["text"].strip()
                            if not text:
                                continue
                            
                            line_text += text + " "
                            font_size = span["size"]
                            max_font_size = max(max_font_size, font_size)
                            
                            if "bold" in span["font"].lower():
                                is_bold = True
                        line_text = line_text.strip()
                        
                    # Detect section headings (adjust threshold based on your PDF)
                    if max_font_size > 11 or is_bold or line_text.isupper():
                        # Check if it's actually a heading (not too long)
                        if len(line_text) < 100 and line_text:
                            # Save previous section
                            if current_section and current_section["content"].strip():
                                sections.append(current_section)
                            
                            # Start new section
                            current_section = {
                                "title": line_text,
                                "content": "",
                                "page_start": page_num + 1,
                                "font_size": max_font_size
                            }
                    elif current_section:
                        current_section["content"] += line_text + "\n"   
                        
        # Add last section
        if current_section and current_section["content"].strip():
            sections.append(current_section)
            
        return sections if sections else None
    
    except Exception as e:
        print(f"   ⚠️  Error fetching PDF section: {str(e)[:60]}")
        return None

def add_chunk_to_collection(collection, doc_type, quarter, chunk_text, chunk_id):
    """
    Add a text chunk to a ChromaDB collection
    
    Args:
        collection: ChromaDB collection object
        doc_type: Type of document (e.g., 'earnings_call')
        quarter: Quarter (Q1, Q2, Q3)
        chunk_text: The text content of the chunk
        chunk_id: Unique identifier for the chunk
    """
    metadata = {
        "doc_type": doc_type,
        "quarter": quarter,
        "chunk_length": len(chunk_text)
    }
    
    collection.add(
        ids=[chunk_id],
        documents=[chunk_text],
        metadatas=[metadata]
    )

def query_collection(collection, doc_type=None, quarter=None):
    """
    Query a ChromaDB collection with optional filters
    
    Args:
        collection: ChromaDB collection object
        doc_type: Optional filter by document type
        quarter: Optional filter by quarter
    
    Returns:
        Query results dict with 'documents', 'metadatas', 'ids'
    """
    where_filter = None
    conditions = []
    
    if doc_type:
        conditions.append({"doc_type": {"$eq": doc_type}})
    if quarter:
        conditions.append({"quarter": {"$eq": quarter}})
    
    if conditions:
        if len(conditions) == 1:
            where_filter = conditions[0]
        else:
            where_filter = {"$and": conditions}
    
    return collection.get(where=where_filter)

def generate_rag_answer(query, chunks, max_chunks=15):
    """
    Generate an answer using GPT-4o based on a query and retrieved chunks
    
    Args:
        query: The user's question
        chunks: List of text chunks to use as context
        max_chunks: Maximum number of chunks to use
    
    Returns:
        Generated answer (string) or None if error
    """
    try:
        # Limit chunks to avoid token overflow
        chunks_to_use = chunks[:max_chunks]
        context = "\n\n".join([f"[Chunk {i+1}]\n{chunk}" for i, chunk in enumerate(chunks_to_use)])
        
        system_prompt = """You are a helpful financial analyst assistant. 
Based on the provided document chunks, answer the user's question accurately and concisely.
If the information is not in the provided chunks, say 'I don't have enough information to answer this question.'
Always cite which chunks you used to answer the question."""
        
        user_message = f"""Question: {query}

Document Chunks:
{context}

Please provide a clear, concise answer based on the above information."""
        
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_message}
            ],
            temperature=0.7
        )
        
        return response.choices[0].message.content
    
    except Exception as e:
        print(f"   ⚠️  Error generating answer: {str(e)[:60]}")
        return None

print("✅ Utility functions defined!")
print()
print("Functions available:")
print("  - fetch_and_extract_pdf(url): Fetch and extract text from PDFs")
print("  - fetch_and_extract_section_from_pdf(url): Extract sections from PDFs using PyMuPDF")
print("  - add_chunk_to_collection(): Add chunks to a collection")
print("  - query_collection(): Query a collection")
print("  - generate_rag_answer(): Generate LLM answers from chunks using GPT-4o")

✅ Utility functions defined!

Functions available:
  - fetch_and_extract_pdf(url): Fetch and extract text from PDFs
  - fetch_and_extract_section_from_pdf(url): Extract sections from PDFs using PyMuPDF
  - add_chunk_to_collection(): Add chunks to a collection
  - query_collection(): Query a collection
  - generate_rag_answer(): Generate LLM answers from chunks using GPT-4o


## Semantic Chunking Approach

In [5]:
import PyPDF2
from langchain_experimental.text_splitter import SemanticChunker
from langchain_openai import OpenAIEmbeddings
from langchain_community.embeddings import HuggingFaceEmbeddings
import re

def extract_semantic_chunk_pdf_from_url(pdf_url, document_type, threshold_amount=None, timeout=10):
    """
    Fetch a PDF from a URL and chunk it based on document type using semantic chunking.
    
    Args:
        pdf_url: URL to the PDF file
        document_type: Type of document - 'press_release', 'transcript', or 'presentation'
        threshold_amount: Optional custom threshold for semantic splitting (uses defaults if None)
        timeout: Request timeout in seconds
    
    Returns:
        List of chunk dictionaries with 'content' and 'metadata', or None if error
    """
    try:
        # Download PDF
        response = requests.get(pdf_url, timeout=timeout)
        response.raise_for_status()
        
        # Extract text from PDF
        pdf_file = BytesIO(response.content)
        pdf_reader = PyPDF2.PdfReader(pdf_file)
        
        text = ""
        for page in pdf_reader.pages:
            text += page.extract_text() + "\n"
        
        text = text.strip()
        
        if not text:
            print(f"   ⚠️  No text extracted from PDF")
            return None
        
        # Chunk the text
        chunks = _chunk_text_by_type(
            text=text,
            document_type=document_type,
            threshold_amount=threshold_amount
        )
        
        # Add PDF URL to metadata
        for chunk in chunks:
            chunk['metadata']['source_url'] = pdf_url
        
        return chunks if chunks else None
        
    except requests.RequestException as e:
        print(f"   ⚠️  Error downloading PDF: {str(e)[:60]}")
        return None
    except Exception as e:
        print(f"   ⚠️  Error chunking PDF: {str(e)[:60]}")
        return None

def _chunk_text_by_type(text, document_type, threshold_amount=None):
    """
    Internal function to chunk text based on document type.
    
    Args:
        text: Text content to chunk
        document_type: Type of document
        threshold_amount: Optional custom threshold
        
    Returns:
        List of chunk dictionaries
    """
    document_type = document_type.lower()
    
    # Validate document type
    valid_types = ['press_release', 'followup_qa', 'earnings_call', 'presentation']
    if document_type not in valid_types:
        print(f"   ⚠️  Invalid document_type: '{document_type}'")
        return None
    
    # Set default thresholds if not provided
    if threshold_amount is None:
        threshold_defaults = {
            'press_release': 90,
            'followup_qa': 85,
            'earnings_call': 85,
            'presentation': 88
        }
        threshold_amount = threshold_defaults[document_type]
    
    # Handle presentation differently (no semantic chunking needed)
    if document_type == 'presentation':
        return _chunk_presentation(text)
    
    # Use semantic chunking for press_release and transcript
    try:
        # from langchain_text_splitters import SemanticChunker
        # from langchain_community.embeddings import HuggingFaceEmbeddings
        
        # embeddings = HuggingFaceEmbeddings(
        #     model_name="sentence-transformers/all-mpnet-base-v2"
        # )
        
        embeddings = OpenAIEmbeddings()
        
        splitter = SemanticChunker(
            embeddings,
            breakpoint_threshold_type="percentile",
            breakpoint_threshold_amount=threshold_amount
        )
        
        chunks = splitter.create_documents([text])
        
        # Format chunks based on document type
        if document_type == 'followup_qa' or document_type == 'earnings_call':
            return _format_transcript_chunks(chunks)
        else:  # press_release
            return _format_standard_chunks(chunks, document_type)
            
    except ImportError as e:
        print(f"   ⚠️  Missing required library: {str(e)[:60]}")
        return None
    except Exception as e:
        print(f"   ⚠️  Error in semantic chunking: {str(e)[:60]}")
        return None

def _chunk_presentation(text, min_slide_length=100):
    """Chunk presentation by splitting on slide indicators"""
    try:
        slide_pattern = r'\n\d+\n'
        slides = re.split(slide_pattern, text)
        
        chunks = []
        for slide_num, slide_content in enumerate(slides, 1):
            if len(slide_content.strip()) > min_slide_length:
                chunks.append({
                    'content': slide_content.strip(),
                    'metadata': {
                        'chunk_id': slide_num - 1,
                        'doc_type': 'presentation',
                        'slide_number': slide_num,
                        'has_chart': 'revenue' in slide_content.lower() or '$' in slide_content,
                        'word_count': len(slide_content.split()),
                        'has_numbers': bool(re.search(r'\$?\d+[\d,]*\.?\d*[BMK]?', slide_content))
                    }
                })
        
        return chunks if chunks else None
        
    except Exception as e:
        print(f"   ⚠️  Error chunking presentation: {str(e)[:60]}")
        return None

def _format_standard_chunks(chunks, doc_type):
    """Format chunks for press releases and other standard documents"""
    try:
        formatted = []
        for i, chunk in enumerate(chunks):
            formatted.append({
                'content': chunk.page_content,
                'metadata': {
                    'chunk_id': i,
                    'doc_type': doc_type,
                    'word_count': len(chunk.page_content.split()),
                    'char_count': len(chunk.page_content),
                    'has_numbers': bool(re.search(r'\$?\d+[\d,]*\.?\d*[BMK]?', chunk.page_content))
                }
            })
        return formatted
        
    except Exception as e:
        print(f"   ⚠️  Error formatting chunks: {str(e)[:60]}")
        return None

def _format_transcript_chunks(chunks):
    """Format chunks for transcripts with speaker detection"""
    try:
        formatted = []
        for i, chunk in enumerate(chunks):
            content = chunk.page_content
            
            # Try to extract speaker name
            speaker_match = re.search(r'^([\w\s]+):', content, re.MULTILINE)
            speaker = speaker_match.group(1).strip() if speaker_match else "Unknown"
            
            # Detect if this is a question
            has_question = bool(re.search(r'\?', content))
            
            formatted.append({
                'content': content,
                'metadata': {
                    'chunk_id': i,
                    'doc_type': 'transcript',
                    'speaker': speaker,
                    'word_count': len(content.split()),
                    'char_count': len(content),
                    'has_numbers': bool(re.search(r'\$?\d+[\d,]*\.?\d*[BMK]?', content)),
                    'has_question': has_question
                }
            })
        
        return formatted
        
    except Exception as e:
        print(f"   ⚠️  Error formatting transcript chunks: {str(e)[:60]}")
        return None

/home/ebby/05-src/personal/gh-ebbypeter/ELVTR-APAC-AISA1-SelfStudy/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Initialize ChromaDB Collections


### Strategy 1 - By Page

By Page chunking splits documents at natural page boundaries, treating each page as a single chunk. 

This approach preserves the complete context within each page, making it ideal for structured documents like financial reports where tables, charts, and related metrics are often grouped on the same page. 

The strategy is simple to implement, deterministic, and ensures that page-level formatting and layout relationships remain intact. 
However, it may include irrelevant content from the same page and can create unnecessarily large chunks if pages contain diverse topics.

#### Build Index

In [6]:
print("📊 Strategy 1 - ByPage: BUILDING INDEX: PAGE-BASED CHUNKING (Each Page = 1 Chunk)")
print("="*70)

s1_bypage_collection = chroma_client.get_or_create_collection(
    name="Strategy1_ByPage_Collection",
    metadata={"description": "Chunks created by treating each PDF page as a separate chunk"}
)

s1_chunk_count = 0
for quarter in data_sources:
    print(f"\n🔄 Processing {quarter}...")
    for doc_type in data_sources[quarter]:
        print(f"  📄 {doc_type}...", end="")
        
        # Fetch PDF data
        pdf_url = data_sources[quarter][doc_type]
        pages = fetch_and_extract_pdf(pdf_url, return_type="Pages")
        
        if not pages:
            print(f"   ⚠️  No text extracted from {pdf_url}")
            continue
        
        # Strategy 1: Each page is one chunk  
        for i, page_text in enumerate(pages):
            chunk_id = f"{quarter}_{doc_type}_s1_page_{i+1}"
            add_chunk_to_collection(s1_bypage_collection, doc_type, quarter, page_text, chunk_id)
            s1_chunk_count += 1
            
        print(f" ✅ {i} pages")
        
print(f"\n✅ Strategy 1: Added {s1_chunk_count} page-based chunks to collection!")

📊 Strategy 1 - ByPage: BUILDING INDEX: PAGE-BASED CHUNKING (Each Page = 1 Chunk)

🔄 Processing Q1...
  📄 press_release... ✅ 8 pages
  📄 presentation... ✅ 17 pages
  📄 earnings_call... ✅ 18 pages
  📄 followup_qa... ✅ 10 pages

🔄 Processing Q2...
  📄 press_release... ✅ 8 pages
  📄 presentation... ✅ 17 pages
  📄 earnings_call... ✅ 18 pages
  📄 followup_qa... ✅ 9 pages

🔄 Processing Q3...
  📄 press_release... ✅ 8 pages
  📄 presentation... ✅ 17 pages
  📄 earnings_call... ✅ 18 pages
  📄 followup_qa... ✅ 9 pages

🔄 Processing Q4...
  📄 press_release... ✅ 8 pages
  📄 presentation... ✅ 17 pages
  📄 earnings_call... ✅ 20 pages
  📄 followup_qa... ✅ 10 pages

✅ Strategy 1: Added 228 page-based chunks to collection!


#### Inspect Index

In [7]:
print("📊 Strategy 1 - ByPage: INSPECT INDEX : PAGE-BASED CHUNKING (Each Page = 1 Chunk)")
print("="*70)

# Get all documents in the collection
s1_all_docs = query_collection(s1_bypage_collection)

print(f"\n✅ Total documents in ByPage Index: {len(s1_all_docs['documents'])}")
print(f"Strategy 1 - ByPage - Statistics:")
s1_chunk_sizes = [m['chunk_length'] for m in s1_all_docs['metadatas']]
print(f"   Average chunk size: {sum(s1_chunk_sizes) / len(s1_chunk_sizes):.0f} characters")
print(f"   Minimum chunk size: {min(s1_chunk_sizes)} characters")
print(f"   Maximum chunk size: {max(s1_chunk_sizes)} characters")

📊 Strategy 1 - ByPage: INSPECT INDEX : PAGE-BASED CHUNKING (Each Page = 1 Chunk)

✅ Total documents in ByPage Index: 228
Strategy 1 - ByPage - Statistics:
   Average chunk size: 2175 characters
   Minimum chunk size: 8 characters
   Maximum chunk size: 5375 characters


### Strategy 2 - By Sentences (3)

By Sentence chunking groups consecutive sentences into fixed-size chunks. In this case, every 3 sentences form one chunk. 

This method provides consistent, small-sized chunks that are easy to process and can capture local context around individual facts or statements. The approach works well for simple factual queries where answers are contained in a few sentences. 

However, it struggles with longer narratives or explanations that span many sentences, often fragmenting content across multiple chunks and making it difficult to retrieve complete context for complex questions.

#### Build Index

In [8]:
print("📊 Strategy 2 - BySentences: BUILDING INDEX: SENTENCE BASED CHUNKING (3 x Sentences = 1 Chunk)")
print("="*70)

s2_bysentences_collection = chroma_client.get_or_create_collection(
    name="Strategy2_BySentences_Collection",
    metadata={"description": "Chunks created by grouping every 3 sentences together"}
)

s2_chunk_count = 0
for quarter in data_sources:
    print(f"\n🔄 Processing {quarter}...")
    for doc_type in data_sources[quarter]:
        print(f"  📄 {doc_type}...", end="")
        
        # Fetch PDF data
        pdf_url = data_sources[quarter][doc_type]
        text = fetch_and_extract_pdf(pdf_url, return_type="Text")
        
        if not text:
            print(f"   ⚠️  No text extracted from {pdf_url}")
            continue
        
        # Strategy 2: Group every 3 sentences into one chunk
        sentences = [s.strip() for s in text.split('.') if s.strip()]
        for i in range(0, len(sentences), 3):
            chunk_sentences = sentences[i:i+3]
            chunk_text = '. '.join(chunk_sentences) + ('.' if chunk_sentences else '')
            chunk_id = f"{quarter}_{doc_type}_s2_sentences_{i//3 + 1}"
            add_chunk_to_collection(s2_bysentences_collection, doc_type, quarter, chunk_text, chunk_id)
            s2_chunk_count += 1
            
        print(f" ✅ {len(sentences)} sentences grouped into {len(sentences) // 3 + (1 if len(sentences) % 3 > 0 else 0)} chunks")
        
print(f"\n✅ Strategy 2: Added {s2_chunk_count} sentence-based chunks to collection!")

📊 Strategy 2 - BySentences: BUILDING INDEX: SENTENCE BASED CHUNKING (3 x Sentences = 1 Chunk)

🔄 Processing Q1...
  📄 press_release... ✅ 108 sentences grouped into 36 chunks
  📄 presentation... ✅ 99 sentences grouped into 33 chunks
  📄 earnings_call... ✅ 460 sentences grouped into 154 chunks
  📄 followup_qa... ✅ 275 sentences grouped into 92 chunks

🔄 Processing Q2...
  📄 press_release... ✅ 120 sentences grouped into 40 chunks
  📄 presentation... ✅ 99 sentences grouped into 33 chunks
  📄 earnings_call... ✅ 459 sentences grouped into 153 chunks
  📄 followup_qa... ✅ 234 sentences grouped into 78 chunks

🔄 Processing Q3...
  📄 press_release... ✅ 144 sentences grouped into 48 chunks
  📄 presentation... ✅ 114 sentences grouped into 38 chunks
  📄 earnings_call... ✅ 443 sentences grouped into 148 chunks
  📄 followup_qa... ✅ 243 sentences grouped into 81 chunks

🔄 Processing Q4...
  📄 press_release... ✅ 128 sentences grouped into 43 chunks
  📄 presentation... ✅ 115 sentences grouped into 39 ch

#### Inspect Index

In [9]:
print("📊 Strategy 2 - BySentence: INSPECT INDEX : SENTENCE BASED CHUNKING (3 x Sentences = 1 Chunk)")
print("="*70)

# Get all documents in the collection
s2_all_docs = query_collection(s2_bysentences_collection)

print(f"\n✅ Total documents in BySentence Index: {len(s2_all_docs['documents'])}")
print(f"Strategy 2 - BySentence - Statistics:")
s2_chunk_sizes = [m['chunk_length'] for m in s2_all_docs['metadatas']]
print(f"   Average chunk size: {sum(s2_chunk_sizes) / len(s2_chunk_sizes):.0f} characters")
print(f"   Minimum chunk size: {min(s2_chunk_sizes)} characters")
print(f"   Maximum chunk size: {max(s2_chunk_sizes)} characters")

📊 Strategy 2 - BySentence: INSPECT INDEX : SENTENCE BASED CHUNKING (3 x Sentences = 1 Chunk)

✅ Total documents in BySentence Index: 1261
Strategy 2 - BySentence - Statistics:
   Average chunk size: 389 characters
   Minimum chunk size: 4 characters
   Maximum chunk size: 4926 characters


### Strategy 3 - By Sections

By Section chunking uses document formatting cues, specifically bold text, larger font sizes, and uppercase text—to detect section headings and split content at these boundaries. 

This approach creates semantically meaningful chunks by preserving complete sections like "CFO Outlook," "Q&A Session," or "Revenue Analysis" as unified chunks. It's highly efficient when documents have clear hierarchical structure with consistent formatting. 

However, it can fail when important content (like financial tables) lacks section headers or when font detection is unreliable, leading to missed or improperly split content.

#### Build Index

In [ ]:
print("📊 Strategy 3 - BySections: BUILDING INDEX: SECTION BASED CHUNKING (1 Section = 1 Chunk)")
print("="*70)

s3_bysections_collection = chroma_client.get_or_create_collection(
    name="Strategy3_BySections_Collection",
    metadata={"description": "Chunks created by treating each section as a separate chunk"}
)

s3_chunk_count = 0
for quarter in data_sources:
    print(f"\n🔄 Processing {quarter}...")
    for doc_type in data_sources[quarter]:
        print(f"  📄 {doc_type}...", end="")
        
        # Fetch PDF data
        pdf_url = data_sources[quarter][doc_type]

        sections = fetch_and_extract_section_from_pdf(pdf_url)
        if not sections or len(sections) == 0:
            print(f"   ⚠️  No sections extracted from {pdf_url}")
            print(f"   Falling back to Sentence-based chunking...")
            pages = fetch_and_extract_pdf(pdf_url, return_type="Pages")
            if not pages:
                print(f"   ⚠️  No text extracted from {pdf_url}")
                continue
            for i, page_text in enumerate(pages):
                chunk_id = f"{quarter}_{doc_type}_s1_page_{i+1}"
                add_chunk_to_collection(s3_bysections_collection, doc_type, quarter, page_text, chunk_id)
                s3_chunk_count += 1
            # Add pages to collection as fallback
            continue
        
        print(f" ✅ Extracted {len(sections) if sections else 0} sections")
        for section in sections:
                chunk_id = f"{quarter}_{doc_type}_s3_section_{s3_chunk_count + 1}"
                add_chunk_to_collection(s3_bysections_collection, doc_type, quarter, section['content'], chunk_id)
                s3_chunk_count += 1
            
print(f"\n✅ Strategy 3: Added {s3_chunk_count} section-based chunks to collection!")

📊 Strategy 3 - BySections: BUILDING INDEX: SECTION BASED CHUNKING (1 Section = 1 Chunk)

🔄 Processing Q1...
  📄 press_release... ✅ Extracted 44 sections
  📄 presentation...   ⚠️  No sections extracted from https://s21.q4cdn.com/399680738/files/doc_financials/2025/q1/Earnings-Presentation-Q1-2025-FINAL.pdf
   Falling back to Sentence-based chunking...
  📄 earnings_call...   ⚠️  No sections extracted from https://s21.q4cdn.com/399680738/files/doc_financials/2025/q1/Transcripts/META-Q1-2025-Earnings-Call-Transcript-1.pdf
   Falling back to Sentence-based chunking...
  📄 followup_qa... ✅ Extracted 11 sections

🔄 Processing Q2...
  📄 press_release... ✅ Extracted 43 sections
  📄 presentation...   ⚠️  No sections extracted from https://s21.q4cdn.com/399680738/files/doc_downloads/Earnings-Presentation-Q2-2025-FINAL.pdf
   Falling back to Sentence-based chunking...
  📄 earnings_call...   ⚠️  No sections extracted from https://s21.q4cdn.com/399680738/files/doc_financials/2025/q2/META-Q2-2025-Ear

#### Inspect Index

In [11]:
print("📊 Strategy 3 - ByParagraphs: INSPECT INDEX : PARAGRAPH BASED CHUNKING (1 Paragraph = 1 Chunk)")
print("="*70)

# Get all documents in the collection
s3_all_docs = query_collection(s3_bysections_collection)
print(f"\n✅ Total documents in BySection Index: {len(s3_all_docs['documents'])}")
print(f"Strategy 3 - BySections - Statistics:")
s3_chunk_sizes = [m['chunk_length'] for m in s3_all_docs['metadatas']]
print(f"   Average chunk size: {sum(s3_chunk_sizes) / len(s3_chunk_sizes):.0f} characters")
print(f"   Minimum chunk size: {min(s3_chunk_sizes)} characters")
print(f"   Maximum chunk size: {max(s3_chunk_sizes)} characters")

📊 Strategy 3 - ByParagraphs: INSPECT INDEX : PARAGRAPH BASED CHUNKING (1 Paragraph = 1 Chunk)

✅ Total documents in BySection Index: 356
Strategy 3 - BySections - Statistics:
   Average chunk size: 1299 characters
   Minimum chunk size: 6 characters
   Maximum chunk size: 38090 characters


### Strategy 4 - Fixed Token Length

By Fixed Token Length chunking divides text into uniform chunks of a predetermined token count (e.g., 512 tokens each), often with overlapping windows to maintain some continuity between chunks. 

This method guarantees predictable chunk sizes, which is useful for managing token limits in language models and ensuring consistent computational costs. 

While simple and deterministic, this approach completely ignores semantic boundaries—it can split sentences mid-way, fragment tables, or separate questions from answers. 
This arbitrary splitting often results in poor retrieval quality.

#### Build Index

In [12]:
print("📊 Strategy 4 - Fixed Token: BUILDING INDEX: FIXED TOKEN CHUNKING (1 Chunk = Fixed Token Length)")
print("="*70)

s4_fixedtoken_collection = chroma_client.get_or_create_collection(
    name="Strategy4_FixedToken_Collection",
    metadata={"description": "Chunks created by splitting text into fixed token lengths (e.g., 1000 tokens per chunk)"}
)

s4_chunk_count = 0
for quarter in data_sources:
    print(f"\n🔄 Processing {quarter}...")
    for doc_type in data_sources[quarter]:
        print(f"  📄 {doc_type}...", end="")
        
        # Fetch PDF data
        pdf_url = data_sources[quarter][doc_type]
        text = fetch_and_extract_pdf(pdf_url, return_type="Text")
        
        if not text:
            print(f"   ⚠️  No text extracted from {pdf_url}")
            continue
        
        # Strategy 4: Split text into fixed token length chunks (e.g., 1000 tokens)
        # For simplicity, we'll approximate tokens by splitting on whitespace and punctuation
        chunk_size = 512  # Approximate number of tokens per chunk
        overlap = 100    # Number of words to overlap between chunks to maintain context
        
        start = 0
        while start < len(text):
             end = start + chunk_size
             chunk_text = text[start:end]
             
             chunk_id = f"{quarter}_{doc_type}_s4_fixedtoken_{s4_chunk_count + 1}"
             add_chunk_to_collection(s4_fixedtoken_collection, doc_type, quarter, chunk_text, chunk_id)
             s4_chunk_count += 1
             start += (chunk_size - overlap)  # Move start forward with overlap
            
        print(f" ✅ {len(text.split())} words split into {len(text.split()) // chunk_size + (1 if len(text.split()) % chunk_size > 0 else 0)} chunks")

📊 Strategy 4 - Fixed Token: BUILDING INDEX: FIXED TOKEN CHUNKING (1 Chunk = Fixed Token Length)

🔄 Processing Q1...
  📄 press_release... ✅ 2519 words split into 5 chunks
  📄 presentation... ✅ 2676 words split into 6 chunks
  📄 earnings_call... ✅ 9344 words split into 19 chunks
  📄 followup_qa... ✅ 5372 words split into 11 chunks

🔄 Processing Q2...
  📄 press_release... ✅ 2974 words split into 6 chunks
  📄 presentation... ✅ 2668 words split into 6 chunks
  📄 earnings_call... ✅ 9622 words split into 19 chunks
  📄 followup_qa... ✅ 4607 words split into 9 chunks

🔄 Processing Q3...
  📄 press_release... ✅ 3277 words split into 7 chunks
  📄 presentation... ✅ 2861 words split into 6 chunks
  📄 earnings_call... ✅ 9682 words split into 19 chunks
  📄 followup_qa... ✅ 4645 words split into 10 chunks

🔄 Processing Q4...
  📄 press_release... ✅ 3019 words split into 6 chunks
  📄 presentation... ✅ 2925 words split into 6 chunks
  📄 earnings_call... ✅ 9995 words split into 20 chunks
  📄 followup_qa...

#### Inspect Index

In [13]:
print("📊 Strategy 4 - Fixed Token: INSPECT INDEX: FIXED TOKEN CHUNKING (1 Chunk = Fixed Token Length)")
print("="*70)

# Get all documents in the collection
s4_all_docs = query_collection(s4_fixedtoken_collection)
print(f"\n✅ Total documents in Fixed Token Index: {len(s4_all_docs['documents'])}")
print(f"Strategy 4 - Fixed Token - Statistics:")
s4_chunk_sizes = [m['chunk_length'] for m in s4_all_docs['metadatas']]
print(f"   Average chunk size: {sum(s4_chunk_sizes) / len(s4_chunk_sizes):.0f} characters")
print(f"   Minimum chunk size: {min(s4_chunk_sizes)} characters")
print(f"   Maximum chunk size: {max(s4_chunk_sizes)} characters")

📊 Strategy 4 - Fixed Token: INSPECT INDEX: FIXED TOKEN CHUNKING (1 Chunk = Fixed Token Length)

✅ Total documents in Fixed Token Index: 1212
Strategy 4 - Fixed Token - Statistics:
   Average chunk size: 508 characters
   Minimum chunk size: 14 characters
   Maximum chunk size: 512 characters


### Strategy 5 - Semantic Chunking

By Semantic Chunking uses embeddings to analyze the semantic similarity between consecutive sentences and identifies natural topic boundaries where similarity drops significantly. 

When the model detects a shift in topic or context, it creates a new chunk, ensuring that semantically related content stays together. This intelligent approach preserves narrative coherence, keeps question-answer pairs intact, and maintains complete explanations without arbitrary splits. It adapts to the content's natural structure rather than imposing fixed rules, making it the most robust strategy across diverse query types—from simple facts to complex narratives—though it requires computational resources for embedding generation.

However, this approach is significantly complicated to implement compared to other strategies and will need to use embeddings.

#### Build Index

In [14]:
print("📊 Strategy 5 - Semantic Chunking: BUILDING INDEX")
print("="*70)

s5_semantic_collection = chroma_client.get_or_create_collection(
    name="Strategy5_SemanticChunking_Collection",
    metadata={"description": "Chunks created using semantic chunking based on content and meaning"}
)

s5_chunk_count = 0
for quarter in data_sources:
    print(f"\n🔄 Processing {quarter}...")
    for doc_type in data_sources[quarter]:
        print(f"  📄 {doc_type}...", end="")
        
        # Fetch PDF data and create semantic chunks
        pdf_url = data_sources[quarter][doc_type]
        chunks = extract_semantic_chunk_pdf_from_url(pdf_url, document_type=doc_type)
        
        if not chunks:
            print(f"   ⚠️  No chunks created from {pdf_url}")
            continue
        
        for chunk in chunks:
            chunk_id = f"{quarter}_{doc_type}_s5_semantic_{s5_chunk_count + 1}"
            add_chunk_to_collection(s5_semantic_collection, doc_type, quarter, chunk['content'], chunk_id)
            s5_chunk_count += 1
            
        print(f" ✅ Created {len(chunks)} semantic chunks")

📊 Strategy 5 - Semantic Chunking: BUILDING INDEX

🔄 Processing Q1...
  📄 press_release... ✅ Created 8 semantic chunks
  📄 presentation... ✅ Created 4 semantic chunks
  📄 earnings_call... ✅ Created 70 semantic chunks
  📄 followup_qa... ✅ Created 45 semantic chunks

🔄 Processing Q2...
  📄 press_release... ✅ Created 9 semantic chunks
  📄 presentation... ✅ Created 4 semantic chunks
  📄 earnings_call... ✅ Created 69 semantic chunks
  📄 followup_qa... ✅ Created 39 semantic chunks

🔄 Processing Q3...
  📄 press_release... ✅ Created 10 semantic chunks
  📄 presentation... ✅ Created 2 semantic chunks
  📄 earnings_call... ✅ Created 68 semantic chunks
  📄 followup_qa... ✅ Created 40 semantic chunks

🔄 Processing Q4...
  📄 press_release... ✅ Created 8 semantic chunks
  📄 presentation... ✅ Created 2 semantic chunks
  📄 earnings_call... ✅ Created 73 semantic chunks
  📄 followup_qa... ✅ Created 43 semantic chunks


#### Inspect Index

In [15]:
print("📊 Strategy 5 - Semantic Chunking: INSPECT INDEX")
print("="*70)

# Get all documents in the collection
s5_all_docs = query_collection(s5_semantic_collection)
print(f"\n✅ Total documents in Semantic Chunking Index: {len(s5_all_docs['documents'])}")
print(f"Strategy 5 - Semantic Chunking - Statistics:")
s5_chunk_sizes = [m['chunk_length'] for m in s5_all_docs['metadatas']]
print(f"   Average chunk size: {sum(s5_chunk_sizes) / len(s5_chunk_sizes):.0f} characters")
print(f"   Minimum chunk size: {min(s5_chunk_sizes)} characters")
print(f"   Maximum chunk size: {max(s5_chunk_sizes)} characters")  

📊 Strategy 5 - Semantic Chunking: INSPECT INDEX

✅ Total documents in Semantic Chunking Index: 494
Strategy 5 - Semantic Chunking - Statistics:
   Average chunk size: 992 characters
   Minimum chunk size: 4 characters
   Maximum chunk size: 16021 characters


### Index Comparison

In [16]:
print(f"\n📊 INDEX COMPARISON:")
print("-" * 70)

print(f"Strategy 1 - ByPage     : Documents = {len(s1_all_docs['documents']):5d} | Avg Size = {sum([m['chunk_length'] for m in s1_all_docs['metadatas']]) / len(s1_all_docs['metadatas']):6.0f} chars")
print(f"Strategy 2 - BySentences: Documents = {len(s2_all_docs['documents']):5d} | Avg Size = {sum([m['chunk_length'] for m in s2_all_docs['metadatas']]) / len(s2_all_docs['metadatas']):6.0f} chars")
print(f"Strategy 3 - BySections : Documents = {len(s3_all_docs['documents']):5d} | Avg Size = {sum([m['chunk_length'] for m in s3_all_docs['metadatas']]) / len(s3_all_docs['metadatas']):6.0f} chars")
print(f"Strategy 4 - FixedToken : Documents = {len(s4_all_docs['documents']):5d} | Avg Size = {sum([m['chunk_length'] for m in s4_all_docs['metadatas']]) / len(s4_all_docs['metadatas']):6.0f} chars")
print(f"Strategy 5 - Semantic   : Documents = {len(s5_all_docs['documents']):5d} | Avg Size = {sum([m['chunk_length'] for m in s5_all_docs['metadatas']]) / len(s5_all_docs['metadatas']):6.0f} chars")


📊 INDEX COMPARISON:
----------------------------------------------------------------------
Strategy 1 - ByPage     : Documents =   228 | Avg Size =   2175 chars
Strategy 2 - BySentences: Documents =  1261 | Avg Size =    389 chars
Strategy 3 - BySections : Documents =   356 | Avg Size =   1299 chars
Strategy 4 - FixedToken : Documents =  1212 | Avg Size =    508 chars
Strategy 5 - Semantic   : Documents =   494 | Avg Size =    992 chars


## Define RAG Prompts

In [17]:
# Define 3 clear RAG prompts
rag_prompts = [
    {
        "id": 1,
        "title": "Simple Fact Question",
        "query": "What was Meta's total revenue in Q2 2025?",
        "search_keywords": ["revenue", "Q2", "total"],
        "doc_types": ["press_release"]
    },
    {
        "id": 2,
        "title": "Long Explanation Question",
        "query": "Explain Mark Zuckerberg's five major AI opportunities and how they relate to Meta's business strategy.",
        "search_keywords": ["AI opportunities", "Zuckerberg", "strategy"],
        "doc_types": ["earnings_call"]
    },
    {
        "id": 3,
        "title": "Comparison Question",
        "query": "Compare Meta's operating margin across Q1, Q2, Q3, and Q4 2025. Which quarter had the highest margin and why?",
        "search_keywords": ["operating margin", "Q1", "Q2", "Q3", "Q4"],
        "doc_types": ["press_release", "presentation"]
    },
    {
        "id": 4,
        "title": "Specific Speaker Question",
        "query": "What did Susan Li say about European regulatory headwinds and DMA compliance in the earnings calls?",
        "search_keywords": ["Susan Li", "DMA", "Europe", "regulatory"],
        "doc_types": ["earnings_call", "followup_qa"]
    },
    {
        "id": 5,
        "title": "Technical Detail Question",
        "query": "How does Meta's GEM (Generative Ads Recommendation model) work and what improvements did it deliver?",
        "search_keywords": ["GEM", "ads recommendation", "model"],
        "doc_types": ["earnings_call", "followup_qa"]
    },
    {
        "id": 6,
        "title": "Future Guidance Question",
        "query": "What is Meta's guidance for Q4 2025 revenue and capital expenditures?",
        "search_keywords": ["guidance", "Q4", "outlook", "capex"],
        "doc_types": ["press_release", "earnings_call"]
    }
]

print("\n" + "="*70)
print("🎯 RAG PROMPTS TO BE TESTED")
print("="*70)

for prompt in rag_prompts:
    print(f"\n📌 Prompt {prompt['id']}: {prompt['title']}")
    print(f"   Question: {prompt['query']}")
    print(f"   Keywords: {', '.join(prompt['search_keywords'])}")
    print(f"   Relevant doc types: {', '.join(prompt['doc_types'])}")


🎯 RAG PROMPTS TO BE TESTED

📌 Prompt 1: Simple Fact Question
   Question: What was Meta's total revenue in Q2 2025?
   Keywords: revenue, Q2, total
   Relevant doc types: press_release

📌 Prompt 2: Long Explanation Question
   Question: Explain Mark Zuckerberg's five major AI opportunities and how they relate to Meta's business strategy.
   Keywords: AI opportunities, Zuckerberg, strategy
   Relevant doc types: earnings_call

📌 Prompt 3: Comparison Question
   Question: Compare Meta's operating margin across Q1, Q2, Q3, and Q4 2025. Which quarter had the highest margin and why?
   Keywords: operating margin, Q1, Q2, Q3, Q4
   Relevant doc types: press_release, presentation

📌 Prompt 4: Specific Speaker Question
   Question: What did Susan Li say about European regulatory headwinds and DMA compliance in the earnings calls?
   Keywords: Susan Li, DMA, Europe, regulatory
   Relevant doc types: earnings_call, followup_qa

📌 Prompt 5: Technical Detail Question
   Question: How does Meta's 

##  Testing Prompts

Run all RAG Prompts against the various indexing strategies and display results

### Strategy 1 - By Page
Page-level segmentation maintaining document structure and pagination

In [18]:
# Execute propmts on Strategy 1 - ByPage Index
print("\n" + "="*70)
print("🚀 EXECUTING RAG PROMPTS ON STRATEGY 1 - BY PAGE INDEX")
print("="*70)

s1_index_results = []
for prompt in rag_prompts:
    print(f"\n🔍 Executing Prompt {prompt['id']} on Strategy 1 - ByPage Index")
    print(f"     Title: {prompt['title']}")
    print(f"     Question: {prompt['query']}")
    
    # Query the collection for relevant chunks based on doc_type and keywords
    retrieved_docs = []
    retrieved_metadata = []
    for doc_type in prompt['doc_types']:
        results = query_collection(s1_bypage_collection, doc_type=doc_type)
        for doc, meta in zip(results['documents'], results['metadatas']):
            if any(keyword.lower() in doc.lower() for keyword in prompt['search_keywords']):
                retrieved_docs.append(doc)
                retrieved_metadata.append(meta)
                
    
    print(f"\n   Retrieved {len(retrieved_docs)} relevant chunks from ByPage Index")
    
    # Generate answer using GPT-4o
    answer = generate_rag_answer(prompt['query'], retrieved_docs)
    s1_index_results.append({
        "prompt_id": prompt['id'],
        "question": prompt['query'],
        "answer": answer,
        "retrieved_chunks": len(retrieved_docs)
    })
    
    print(f"💬 Generated Answer:\n{answer}")
    
    print(f"\n   Top 3 Retrieved Chunks of {len(retrieved_docs)}:")
    for i, chunk in enumerate(retrieved_docs[:3]):
        print(f"   => Chunk ID: {results['ids'][i]}")
        print(f"      Type: {retrieved_metadata[i]['doc_type']} | Quarter: {retrieved_metadata[i]['quarter']} | Chunk Length: {retrieved_metadata[i]['chunk_length']} characters")
        print(f"      Data: {chunk[:100]}\n")
    print("-" * 70)
        


🚀 EXECUTING RAG PROMPTS ON STRATEGY 1 - BY PAGE INDEX

🔍 Executing Prompt 1 on Strategy 1 - ByPage Index
     Title: Simple Fact Question
     Question: What was Meta's total revenue in Q2 2025?

   Retrieved 32 relevant chunks from ByPage Index
💬 Generated Answer:
Meta's total revenue in Q2 2025 was $47.516 billion. [Chunk 9]

   Top 3 Retrieved Chunks of 32:
   => Chunk ID: Q1_press_release_s1_page_1
      Type: press_release | Quarter: Q1 | Chunk Length: 2418 characters
      Data: Meta Reports First Quarter 2025  Results
MENLO PARK, Calif. – April 30, 2025  – Meta Platforms, Inc.

   => Chunk ID: Q1_press_release_s1_page_2
      Type: press_release | Quarter: Q1 | Chunk Length: 1768 characters
      Data: CFO Outlook Commentary
We expect second quarter 2025 total revenue to be in the range of $ 42.5-45.5

   => Chunk ID: Q1_press_release_s1_page_3
      Type: press_release | Quarter: Q1 | Chunk Length: 5346 characters
      Data: Forward-Looking Statements
This press release conta

### Strategy 2 - By Sentences (3)

N-sentence windowing approach (n=3) for consistent chunk granularity

In [19]:
# Execute prompts on Strategy 2 - BySentences Index
print("\n" + "="*70)
print("🚀 EXECUTING RAG PROMPTS ON STRATEGY 2 - BY SENTENCES INDEX")
print("="*70)

s2_index_results = []
for prompt in rag_prompts:
    print(f"\n🔍 Executing Prompt {prompt['id']} on Strategy 2 - BySentences Index")
    print(f"     Title: {prompt['title']}")
    print(f"     Question: {prompt['query']}")
    
    # Query the collection for relevant chunks based on doc_type and keywords
    retrieved_docs = []
    retrieved_metadata = []
    for doc_type in prompt['doc_types']:
        results = query_collection(s2_bysentences_collection, doc_type=doc_type)
        for doc, meta in zip(results['documents'], results['metadatas']):
            if any(keyword.lower() in doc.lower() for keyword in prompt['search_keywords']):
                retrieved_docs.append(doc)
                retrieved_metadata.append(meta)
                
    
    print(f"\n   Retrieved {len(retrieved_docs)} relevant chunks from BySentences Index")
    
    # Generate answer using GPT-4o
    answer = generate_rag_answer(prompt['query'], retrieved_docs)
    s2_index_results.append({
        "prompt_id": prompt['id'],
        "question": prompt['query'],
        "answer": answer,
        "retrieved_chunks": len(retrieved_docs)
    })
    
    print(f"💬 Generated Answer:\n{answer}")
    
    print(f"\n   Top 3 Retrieved Chunks of {len(retrieved_docs)}:")
    for i, chunk in enumerate(retrieved_docs[:3]):
        print(f"   => Chunk ID: {results['ids'][i]}")
        print(f"      Type: {retrieved_metadata[i]['doc_type']} | Quarter: {retrieved_metadata[i]['quarter']} | Chunk Length: {retrieved_metadata[i]['chunk_length']} characters")
        print(f"      Data: {chunk[:100]}\n")
    print("-" * 70)


🚀 EXECUTING RAG PROMPTS ON STRATEGY 2 - BY SENTENCES INDEX

🔍 Executing Prompt 1 on Strategy 2 - BySentences Index
     Title: Simple Fact Question
     Question: What was Meta's total revenue in Q2 2025?

   Retrieved 56 relevant chunks from BySentences Index
💬 Generated Answer:
Meta's total revenue in Q2 2025 was $47,516 million. [Chunk 14]

   Top 3 Retrieved Chunks of 56:
   => Chunk ID: Q1_press_release_s2_sentences_1
      Type: press_release | Quarter: Q1 | Chunk Length: 717 characters
      Data: "We've had a strong start to an important year, our community continues to grow and our business is 

   => Chunk ID: Q1_press_release_s2_sentences_2
      Type: press_release | Quarter: Q1 | Chunk Length: 208 characters
      Data: •Ad impressions – Ad impressions delivered across our Family of Apps increased  by 5% year-over-year

   => Chunk ID: Q1_press_release_s2_sentences_3
      Type: press_release | Quarter: Q1 | Chunk Length: 183 characters
      Data: 31 billion , an increas

### Strategy 3 - By Sections

Typography-based segmentation using font properties as hierarchical markers

In [20]:
# Execute prompts on Strategy 3 - BySections Index
print("\n" + "="*70)
print("🚀 EXECUTING RAG PROMPTS ON STRATEGY 3 - BY SECTIONS INDEX")
print("="*70)

s3_index_results = []
for prompt in rag_prompts: 
    print(f"\n🔍 Executing Prompt {prompt['id']} on Strategy 3 - BySections Index")
    print(f"     Title: {prompt['title']}")
    print(f"     Question: {prompt['query']}")
    
    # Query the collection for relevant chunks based on doc_type and keywords
    retrieved_docs = []
    retrieved_metadata = []
    for doc_type in prompt['doc_types']:
        results = query_collection(s3_bysections_collection, doc_type=doc_type)
        for doc, meta in zip(results['documents'], results['metadatas']):
            if any(keyword.lower() in doc.lower() for keyword in prompt['search_keywords']):
                retrieved_docs.append(doc)
                retrieved_metadata.append(meta)
                
    
    print(f"\n   Retrieved {len(retrieved_docs)} relevant chunks from BySections Index")
    
    # Generate answer using GPT-4o
    answer = generate_rag_answer(prompt['query'], retrieved_docs)
    s3_index_results.append({
        "prompt_id": prompt['id'],
        "question": prompt['query'],
        "answer": answer,
        "retrieved_chunks": len(retrieved_docs)
    })
    
    print(f"💬 Generated Answer:\n{answer}")
    
    print(f"\n   Top 3 Retrieved Chunks of {len(retrieved_docs)}:")
    for i, chunk in enumerate(retrieved_docs[:3]):
        print(f"   => Chunk ID: {results['ids'][i]}")
        print(f"      Type: {retrieved_metadata[i]['doc_type']} | Quarter: {retrieved_metadata[i]['quarter']} | Chunk Length: {retrieved_metadata[i]['chunk_length']} characters")
        print(f"      Data: {chunk[:100]}\n")
    print("-" * 70)


🚀 EXECUTING RAG PROMPTS ON STRATEGY 3 - BY SECTIONS INDEX

🔍 Executing Prompt 1 on Strategy 3 - BySections Index
     Title: Simple Fact Question
     Question: What was Meta's total revenue in Q2 2025?

   Retrieved 41 relevant chunks from BySections Index
💬 Generated Answer:
Meta's total revenue in Q2 2025 was $42,314 million. [Chunk 1]

   Top 3 Retrieved Chunks of 41:
   => Chunk ID: Q1_press_release_s3_section_1
      Type: press_release | Quarter: Q1 | Chunk Length: 305 characters
      Data: Revenue
$
42,314 $
36,455
16 %
Costs and expenses

24,759
22,637
9 %
Income from operations
$
17,555

   => Chunk ID: Q1_press_release_s3_section_2
      Type: press_release | Quarter: Q1 | Chunk Length: 1731 characters
      Data: We expect second quarter 2025 total revenue to be in the range of $42.5-45.5 billion. Our guidance a

   => Chunk ID: Q1_press_release_s3_section_3
      Type: press_release | Quarter: Q1 | Chunk Length: 2613 characters
      Data: This press release contains for

### Strategy 4 - Fixed Token Length

Token-count-based chunking with configurable size and overlap windows

In [21]:
# Execute prompts on Strategy 4 - Fixed Token Index
print("\n" + "="*70)
print("🚀 EXECUTING RAG PROMPTS ON STRATEGY 4 - FIXED TOKEN INDEX")
print("="*70)

s4_index_results = []
for prompt in rag_prompts:
    print(f"\n🔍 Executing Prompt {prompt['id']} on Strategy 4 - Fixed Token Index")
    print(f"     Title: {prompt['title']}")
    print(f"     Question: {prompt['query']}")
    
    # Query the collection for relevant chunks based on doc_type and keywords
    retrieved_docs = []
    retrieved_metadata = []
    for doc_type in prompt['doc_types']:
        results = query_collection(s4_fixedtoken_collection, doc_type=doc_type)
        for doc, meta in zip(results['documents'], results['metadatas']):
            if any(keyword.lower() in doc.lower() for keyword in prompt['search_keywords']):
                retrieved_docs.append(doc)
                retrieved_metadata.append(meta)
                
    
    print(f"\n   Retrieved {len(retrieved_docs)} relevant chunks from Fixed Token Index")
    
    # Generate answer using GPT-4o
    answer = generate_rag_answer(prompt['query'], retrieved_docs)
    s4_index_results.append({
        "prompt_id": prompt['id'],
        "question": prompt['query'],
        "answer": answer,
        "retrieved_chunks": len(retrieved_docs)
    })
    
    print(f"💬 Generated Answer:\n{answer}")
    
    print(f"\n   Top 3 Retrieved Chunks of {len(retrieved_docs)}:")
    for i, chunk in enumerate(retrieved_docs[:3]):
        print(f"   => Chunk ID: {results['ids'][i]}")
        print(f"      Type: {retrieved_metadata[i]['doc_type']} | Quarter: {retrieved_metadata[i]['quarter']} | Chunk Length: {retrieved_metadata[i]['chunk_length']} characters")
        print(f"      Data: {chunk[:100]}\n")
    print("-" * 70)


🚀 EXECUTING RAG PROMPTS ON STRATEGY 4 - FIXED TOKEN INDEX

🔍 Executing Prompt 1 on Strategy 4 - Fixed Token Index
     Title: Simple Fact Question
     Question: What was Meta's total revenue in Q2 2025?

   Retrieved 89 relevant chunks from Fixed Token Index
💬 Generated Answer:
I don't have enough information to answer this question. The document chunks do not provide Meta's total revenue for Q2 2025; they only mention the expected revenue range for that quarter.

   Top 3 Retrieved Chunks of 89:
   => Chunk ID: Q1_press_release_s4_fixedtoken_1
      Type: press_release | Quarter: Q1 | Chunk Length: 512 characters
      Data: which now has 
almost 1 billion monthly actives."
First Quarter 2025  Financial Highlights
Three Mon

   => Chunk ID: Q1_press_release_s4_fixedtoken_2
      Type: press_release | Quarter: Q1 | Chunk Length: 512 characters
      Data:  %  13 %
Net income $ 16,644 $ 12,369  35 %
Diluted earnings per share (EPS) $ 6.43 $ 4.71  37 %
Fir

   => Chunk ID: Q1_press_rel

### Strategy 5 - Semantic Chunking

Embedding-similarity-based segmentation identifying topical coherence boundaries

In [22]:
# Execute prompts on Strategy 5 - Semantic Chunking Index
print("\n" + "="*70)
print("🚀 EXECUTING RAG PROMPTS ON STRATEGY 5 - SEMANTIC CHUNKING")
print("="*70)

s5_index_results = []
for prompt in rag_prompts:
    print(f"\n🔍 Executing Prompt {prompt['id']} on Strategy 5 - Semantic Chunking Index")
    print(f"     Title: {prompt['title']}")
    print(f"     Question: {prompt['query']}")
    
    # Query the collection for relevant chunks based on doc_type and keywords
    retrieved_docs = []
    retrieved_metadata = []
    for doc_type in prompt['doc_types']:
        results = query_collection(s5_semantic_collection, doc_type=doc_type)
        for doc, meta in zip(results['documents'], results['metadatas']):
            if any(keyword.lower() in doc.lower() for keyword in prompt['search_keywords']):
                retrieved_docs.append(doc)
                retrieved_metadata.append(meta)
                
    
    print(f"\n   Retrieved {len(retrieved_docs)} relevant chunks from Semantic Chunking Index")
    
    # Generate answer using GPT-4o
    answer = generate_rag_answer(prompt['query'], retrieved_docs)
    s5_index_results.append({
        "prompt_id": prompt['id'],
        "question": prompt['query'],
        "answer": answer,
        "retrieved_chunks": len(retrieved_docs)
    })
    
    print(f"💬 Generated Answer:\n{answer}")
    
    print(f"\n   Top 3 Retrieved Chunks of {len(retrieved_docs)}:")
    for i, chunk in enumerate(retrieved_docs[:3]):
        print(f"   => Chunk ID: {results['ids'][i]}")
        print(f"      Type: {retrieved_metadata[i]['doc_type']} | Quarter: {retrieved_metadata[i]['quarter']} | Chunk Length: {retrieved_metadata[i]['chunk_length']} characters")
        print(f"      Data: {chunk[:100]}\n")
    print("-" * 70)


🚀 EXECUTING RAG PROMPTS ON STRATEGY 5 - SEMANTIC CHUNKING

🔍 Executing Prompt 1 on Strategy 5 - Semantic Chunking Index
     Title: Simple Fact Question
     Question: What was Meta's total revenue in Q2 2025?

   Retrieved 25 relevant chunks from Semantic Chunking Index
💬 Generated Answer:
Meta's total revenue in Q2 2025 was $47.516 billion. 

This information was found in Chunk 7.

   Top 3 Retrieved Chunks of 25:
   => Chunk ID: Q1_press_release_s5_semantic_1
      Type: press_release | Quarter: Q1 | Chunk Length: 1208 characters
      Data: Meta Reports First Quarter 2025  Results
MENLO PARK, Calif. – April 30, 2025  – Meta Platforms, Inc.

   => Chunk ID: Q1_press_release_s5_semantic_2
      Type: press_release | Quarter: Q1 | Chunk Length: 2318 characters
      Data: •Average price per ad – Average price per ad increased  by 10% year-over-year. •Revenue – Revenue wa

   => Chunk ID: Q1_press_release_s5_semantic_3
      Type: press_release | Quarter: Q1 | Chunk Length: 4523 chara

## Evaluation

### Analysis By Question

#### 📌 Question 1: Simple Fact Question
**"What was Meta's total revenue in Q2 2025?"**

| Strategy | Answered? | Chunks Retrieved | Accuracy | Notes |
|----------|----------------|------------------|----------|-------|
| **Semantic Chunking** | Yes | 25 | **Correct** ($47.516B + YoY context) | Best - fewest chunks, most context |
| **By Page** | Yes | 32 | **Correct** ($47.516B) | Good but retrieved more chunks |
| **By Sentences** | Yes | 56 | **Correct** ($47,516M) | Too many chunks for simple fact |
| **By Sections** | No | 41 | **WRONG** | "Not enough information" |
| **Fixed Token** | No | 89 | **WRONG** | "Not enough information" |

**Winner: Semantic Chunking** - Most efficient (25 chunks vs 32-89), added context (YoY growth)

**Key Insight**: Fixed Token and By Sections likely split the financial table, losing critical context.

---

#### 📌 Question 2: Long Explanation Question
**"Explain Mark Zuckerberg's five major AI opportunities and business strategy"**

| Strategy | Answered? | Chunks Retrieved | Completeness | Notes |
|----------|----------------|------------------|--------------|-------|
| **Semantic Chunking** | Yes | 36 | **Complete** (All 5 explained + strategy) | Best Response |
| **By Sections** | Yes | 24 | **Complete** (All 5 explained + strategy) | Fewest chunks, excellent quality |
| **By Page** | Yes | 30 | **Complete** (All 5 explained) | Good but slightly more verbose |
| **By Sentences** | No | 37 | **WRONG** | "Not enough information" |
| **Fixed Token** | No | 45 | **WRONG** | "Not enough information" |

**Winner: By Sections (24 chunks)** with Semantic Chunking close second (36 chunks)

**Key Insight**: Long narrative needs semantic boundaries. Sentence-based chunking fragments the narrative. Fixed Token splits mid-explanation.

---

#### 📌 Question 3: Comparison Question
**"Compare operating margins Q1-Q4 2025, which was highest and why?"**

| Strategy | Answered? | Chunks Retrieved | Accuracy | Explanation Quality |
|----------|----------------|------------------|----------|---------------------|
| **Semantic Chunking** | Yes | 16 | **All Correct** | Clear explanation | 
| **By Page** | Yes | 64 | **All Correct** | Clear explanation |
| **By Sections** | Yes | 64 | **Q4 wrong (42% vs 41%)** | Minor error |
| **By Sentences** | Yes | 39 | **All Correct** | Clear explanation |
| **Fixed Token** | Yes | 71 | **All Correct** | Good explanation |

**Winner: Semantic Chunking** - Most efficient (only 16 chunks for 4-quarter comparison!)

**Key Insight**: All strategies handled this well. Semantic Chunking's efficiency is remarkable - 4x fewer chunks than other options.

---

#### 📌 Question 4: Specific Speaker Question
**"What did Susan Li say about European regulatory headwinds and DMA?"**

| Strategy | Answered? | Chunks Retrieved | Accuracy | Context Preserved |
|----------|----------------|------------------|----------|-------------------|
| **Semantic Chunking** | Yes | 98 | **Complete** | Full context + revenue impact (16%) |
| **By Page** | Yes | 82 | **Complete** | Good context |
| **By Sections** | Yes | 70 | **Complete** | Good context |
| **Fixed Token** | Yes | 137 | **Complete** | Good but too many chunks |
| **By Sentences** | No | 106 | **ERROR** | Rate limited (429 error) |

**Winner: By Sections (70 chunks)** - Most efficient while maintaining quality

**Key Insight**: Sentence-based chunking retrieved too many fragments and hit rate limits. Semantic Chunking provided most comprehensive answer with extra revenue context.

---

#### 📌 Question 5: Technical Detail Question
**"How does GEM model work and what improvements did it deliver?"**

| Strategy | Answered? | Chunks Retrieved | Completeness | Technical Detail |
|----------|----------------|------------------|--------------|------------------|
| **Semantic Chunking** | Yes | 101 | **Complete** | Full technical explanation |
| **By Page** | Yes | 85 | **Complete** | Full technical explanation |
| **By Sections** | Yes | 70 | **Complete** | Full technical explanation |
| **By Sentences** | Yes | 183 | **Partial** | Basic explanation only |
| **Fixed Token** | No | 236 | **WRONG** | "Not enough information" |

**Winner: By Sections (70 chunks)** - Most efficient with complete answer

**Key Insight**: Fixed Token retrieved 236 chunks but still failed! Technical explanations need semantic boundaries. Sentence chunking retrieved too many fragments.

---

#### 📌 Question 6: Future Guidance Question
**"What is Meta's Q4 2025 revenue and CapEx guidance?"**

| Strategy | Answered? | Chunks Retrieved | Accuracy | Notes |
|----------|----------------|------------------|----------|-------|
| **Semantic Chunking** | Yes | 51 | **Correct** (Revenue + CapEx) | Complete answer |
| **By Page** | Yes | 39 | **Correct** (Revenue + CapEx) | Most efficient |
| **By Sections** | Yes | 28 | **Correct** (Revenue + CapEx) | Most efficient! |
| **By Sentences** | Yes | 95 | **Correct** (Revenue + CapEx) | Too many chunks |
| **Fixed Token** | Yes | 105 | **Correct** (Revenue + CapEx) | Too many chunks |

**Winner: By Sections (28 chunks)** - Most efficient

**Key Insight**: All strategies succeeded here. "CFO Outlook" sections are well-structured, making this easier.


### Summary

| Strategy | Questions Answered | Success Rate | Avg Chunks Retrieved | Comments |
|----------|-------------------|--------------|---------------------|------------|
| **Semantic Chunking** | 6/6 | **100%** | 54.5 | Overall Best |
| **By Page** | 6/6 | **100%** | 55.2 | Strong Second |
| **By Sections** | 5/6 | **83%** | 49.5 | Efficient, but risky based on document type |
| **By Sentences** | 5/6 | **83%** | 86.0 | Not recommended |
| **Fixed Token** | 3/6 | **50%** | 120.8 | Poor Performance |

### Key Insights

1. **Semantic Chunking** - Strong Option
- 100% success rate
- Balanced efficiency vs. accuracy trade-off
- More complicated than other strategies considered here

2. **By Page** - Safe & Simple
- 100% success rate
- Simple implementation
- Good default if Semantic Chunking cannot be implemented.

3. **By Section** - Has a critical flaw
- Extremely efficient when it works
- Fails on tables/metrics or when document is not hierarchical (presentation)
- Need a fallback approach for some document types. Here we used ByPage strategy if we could not determine sections in the document.

4. **By Sentences** - Chunking fragments too much
- 183 chunks for the techical question
- Caused rate limiting in some cases

5. **Fixed Token Length** - False Economy
- Retrieved most chunks compared to other strategies
- Failed 50% of questions
- Arbitrarily split the documents.